# Torchvision object detection finetuning
> 튜토리얼 코드 유지보수가 되지 않아 실행은 되지 않는다.

사전학습된 모델의 미세조정을 해 보자. 예제로서 [Penn-Fudan 데이터셋](https://www.cis.upenn.edu/~jshi/ped_html/)으로 사전학습된 Mask R-CNN을 사용한다. 여기에는 170개 이미지와 345개 보행자 인스턴스가 포함되어 있다. 커스텀 데이터셋으로 인스턴트 분류 모델을 학습시킬 것이다.

## Defining the Dataset
커스텀 데이터셋은 `torch.utils.data.Dataset` 클래스를 상속해야 하며, 함수 `__len__()`과 `__getitem__()`을 구현해야 한다. 특히 `__getitem__()`은 이하를 반환해야 한다.

- image: 크기 (H, W)인 PIL(Python Imaging Library) 이미지
- target: 이하의 필드를 갖는 딕셔너리
    - boxes: 바운딩 박스의 모서리 좌표 (x0, y0, x1, y1), FloatTensor[N, 4]
    - labels: 각 박스의 레이블, Int64Ternsor[N], 배경 레이블은 0으로 약속되어 있다.
    - image_id: 이미지 식별자, Int64Tensor[1]
    - area: 바운딩 박스의 면적, Tensor[N]
    - iscrowd: 복수의 객체가 포함되었는지 여부, 평가 시 제외, UInt8Tensor[N]
    - (optionally) masks: 각 객체에 대한 분류 마스크, UInt8Tensor[N, H, W]
    - (optionally) keypoints: 각 객체에 대한 K개 키포인트 (x, y, visibility), FloatTensor[N, K, 3]

종횡비가 비슷한 이미지들로 배치를 구성하려면 함수 `get_height_and_width()`를 추가해야 한다.

### Writing a custom dataset for PennFudan
PennFudan 파일을 [다운로드](https://www.cis.upenn.edu/~jshi/ped_html/PennFudanPed.zip)하자. 그리고 여기에 맞는 데이터셋을 작성한다.